#### [Weights and Biases](https://www.wandb.com/) is a tool for visualizing and tracking machine learning experiments.

In [ ]:
# Import abilities
from ai import Chat, Vision, Audio, Images, Embeddings, FineTuner
import wandb
import time
from wandb_logging import WandbSpanManager

# Adjust model globally here or modify in function calls. Vision model cannot be changed, for now ;) 
model = "gpt-3.5-turbo-1106"

### Weights and Biases x OpenAI-Launchpad

Using wandb prompts and traces to visualize the flow in AI systems.

In [ ]:
wb = WandbSpanManager(name="system")

wandb.init(project="openai-launchpad") ## Initialize wandb project

audio = Audio()
chat = Chat(model=model, system="Helpful assistant.")

chain_name = "conversation"

agent_name = "agent"

agent_span = wb.wandb_span(span_kind = "LLM", span_name=agent_name, parent_span_id=None)

for i in range(1):

    chain_span = wb.wandb_span(span_kind = "chain", span_name=chain_name, parent_span_id=agent_span)

    # transcript = audio.record_and_transcribe()
    transcript = "This is a test."
    time.sleep(1)

    transcript_span = wb.wandb_span(
        span_kind="tool",
        span_name="transcription",
        parent_span_id=chain_span,
        inputs={"audio": "audio"},
        outputs={"transcript": transcript},
    )

    if "quit" in transcript:
        break

    completion, messages = chat.chat_completion(transcript, speak=False, stream=True, memories=True)

    # Log the LLM used.
    llm_span = wb.wandb_span(
        span_kind="llm",
        span_name="chat",
        parent_span_id = chain_span,
        inputs={"transcript": transcript},
        outputs={"completion": completion, "messages": messages},
        metadata={"model": model},
    )

    voice = "echo"
    # audio.speak(completion, voice=voice)
    time.sleep(2)

    # Log the speech span.
    speech_span = wb.wandb_span(
        span_kind="tool",
        span_name="speech",
        parent_span_id = llm_span,
        inputs={"completion": completion},
        outputs={"tool": "Completion spoken."},
        metadata={"voice": voice}
    )

wb.update_span_by_id(agent_span, inputs={"transcript": transcript, "test": "test"}, outputs={"completion": completion, "messages": messages})

wb.log_top_level_span()

# End wandb run
wandb.finish()

In [ ]:
wb = WandbSpanManager(name="image_generation_and_vision")

# Initialize wandb project
wandb.init(project="openai-launchpad")

image_prompt = "The power of time in the palm of your hand."

# Start the chain for logging
chain_span = wb.wandb_span(span_kind="chain", span_name="image_generation_and_vision", parent_span_id=None)

# Generate an image
images = Images()
response, path = images.generate_image(image_prompt, display_image=True, save_image=True)

# Log the image generation span
image_gen_span = wb.wandb_span(
    span_kind="tool",
    span_name="image_generation",
    inputs={"prompt": image_prompt},
    outputs={"response": response, "image_path": path},
    parent_span_id=chain_span,
    metadata={"display_image": True, "save_image": True}
)

# Vision system analysis
vision = Vision(system="You can only respond with bullet points.")
response, messages = vision.vision_completion(prompt="What is this image showing?", image_paths=[path], memories=True, stream=True)

# Log the vision system span
vision_span = wb.wandb_span(
    span_kind="llm",
    span_name="vision_description",
    inputs={"prompt": "What is this image showing?", "image_paths": [path]},
    outputs={"response": response, "messages": messages},
    parent_span_id=chain_span
)

wb.log_top_level_span()

# End wandb run
wandb.finish()